# Working with Edgar datasets: Wrangling, Pre-processing and exploratory data analysis

In [1]:
#!pip install requests

In [2]:
#!pip install pandas

In [3]:
#!pip install lxml

In [14]:
#!pip install beautifulsoup4

In [27]:
from lxml import html
from lxml.html import parse
import requests
import html5lib
#from BeautifulSoup import BeautifulSoup
import pandas as pd
from pandas.io.parsers import TextParser
from urllib.request import urlopen
from bs4 import BeautifulSoup
import csv
import sys

Input variables (need to set as environmental variables for Docker pipline)

In [18]:
cik = '51143'
acc_no = '0000051143-13-000007'
acc_short = acc_no[0:10] + acc_no[11:13] + acc_no[14:]
#print(acc_short)   

Generate full URL and get the page

In [19]:
page = requests.get('https://www.sec.gov/Archives/edgar/data/' + cik + '/' + acc_short + '/' + acc_no +'-index.htm')
source = page.content
#print(source)

Scrape page content for links, find 10Q

In [20]:
htmlcontent = html.document_fromstring(source)
#print(htmlcontent)

In [21]:
href = ''
for elem in htmlcontent.iterlinks():
    #print(elem[2])
    if "10q.htm" in elem[2]:
        href+= elem[2]

In [22]:
url10q = 'https://www.sec.gov' + href
print(url10q)

https://www.sec.gov/Archives/edgar/data/51143/000005114313000007/ibm13q3_10q.htm


### Attempt based on class code

In [10]:
parsed = parse(urlopen(url10q))
doc = parsed.getroot()

In [11]:
tables = doc.findall('.//table')
example = tables[6]
#print(example)

In [12]:
def _unpack(row, kind='td'):
    elements = row.findall('.//%s' % kind)
    return [val.text_content() for val in elements]

#rows = example.findall('.//tr')
#for row in rows:
#    print(_unpack(row, kind='td'))
#data = [_unpack(row, kind='td') for row in rows]    
#print(data)

In [13]:
def parse_options_data(table):
    rows = table.findall('.//tr')
    header = _unpack(rows[0], kind='th')
    if len(header) > 0:
        data = [_unpack(r) for r in rows[1:]] 
    else: 
        data = [_unpack(r) for r in rows]     
    print(len(data))
    #for r in data:
    #    print(r)
    return TextParser(data, names=header).get_chunk()
    
example_data = parse_options_data(example)
#print(example_data)

35


TypeError: object of type 'NoneType' has no len()

### Attempt based on BeautifulSoup

In [28]:
soup = BeautifulSoup(requests.get(url10q).text, 'lxml')
csvout  = csv.writer(sys.stdout)

In [ ]:
def unicode_cleanup():
    

In [54]:
tables=soup.findAll('table')
#print(tables[1])
table_count=0

In [57]:
for table in tables:
    rows = table.findAll('tr')
    clean_table = [] 
    
    for row in rows:
        clean_row = []
        columns = row.find_all('td')
        for column in columns:
            clean_row.append(column.get_text().replace("\n","").replace("\xa0",""))
        clean_table.append(clean_row)
        #print(clean_row)
    frame = pd.DataFrame(clean_table)
    #print(frame)
    csv_name = acc_no + "_" + str(table_count) + ".csv"
    frame.to_csv(csv_name)
    table_count +=1